# V1 Database Changes (Databricks)

Applies schema/runtime deltas that are newer than the legacy schema notebook.
This includes additional runtime tables, reporting view updates, and optimize statements.

In [ ]:
dbutils.widgets.text('catalog', 'vendorcat_dev')
dbutils.widgets.text('schema', 'vendorcat_v1')
dbutils.widgets.text('sql_root', '/Workspace/Repos/PrideRock-CoPilot/VendorCat/setup/v1_schema/databricks')

catalog = dbutils.widgets.get('catalog').strip()
schema = dbutils.widgets.get('schema').strip()
sql_root = dbutils.widgets.get('sql_root').strip()

assert catalog, 'catalog parameter is required'
assert schema, 'schema parameter is required'
assert sql_root, 'sql_root parameter is required'

spark.sql(f"USE CATALOG `{catalog}`")
spark.sql(f"USE SCHEMA `{schema}`")
print(f'Applying database changes for catalog={catalog} schema={schema}')
print(f'SQL root: {sql_root}')

In [ ]:
import re
from pathlib import Path

token_pattern = re.compile(r'\$\{(CATALOG|SCHEMA)\}')

def render_sql(sql_text: str, catalog_name: str, schema_name: str) -> str:
    context = {'CATALOG': catalog_name, 'SCHEMA': schema_name}
    return token_pattern.sub(lambda m: context[m.group(1)], sql_text)

def execute_sql_script(file_path: str) -> None:
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f'SQL file not found: {file_path}')
    raw = path.read_text(encoding='utf-8')
    rendered = render_sql(raw, catalog, schema)
    statements = [stmt.strip() for stmt in rendered.split(';') if stmt.strip()]
    for statement in statements:
        spark.sql(statement)
    print(f'Applied {path.name} ({len(statements)} statements)')

In [ ]:
schema_files = [
    f'{sql_root}/05_create_functional_parity_bridge.sql',
    f'{sql_root}/06_create_functional_runtime_compat.sql',
    f'{sql_root}/07_create_reporting_views.sql',
    f'{sql_root}/90_create_indexes.sql',
]

for sql_file in schema_files:
    execute_sql_script(sql_file)

print('Database changes applied.')

In [ ]:
required_tables = [
    'app_vendor_warning',
    'app_offering_payment',
    'app_import_job',
    'app_import_stage_row',
    'app_import_stage_payment',
]

for table_name in required_tables:
    exists = spark.sql(
        f"SHOW TABLES IN `{catalog}`.`{schema}` LIKE '{table_name}'"
    ).count() > 0
    status = 'OK' if exists else 'MISSING'
    print(f"{table_name}: {status}")

view_exists = spark.sql(
    f"SHOW VIEWS IN `{catalog}`.`{schema}` LIKE 'vw_employee_directory'"
).count() > 0
view_status = 'OK' if view_exists else 'MISSING'
print(f"vw_employee_directory view: {view_status}")